# **Introducción al Cálculo: Funciones, derivadas y algo más**

En este tutorial es una introducción al **Cálculo**, principalmente el concepto de derivadas, dando algunas definiciones claves del área y tratando de ir anticipando su relevancia en **Machine Learning** y **Deep Learning**. Asumiremos que no tienen un conocimiento del área pero si que estan minimamente familiarizados con los conceptos básicos de **función** y la mátematica necesaria de un curso de pregrado.

¿Que **SÍ** busca esta clase?:

* Construir una intuición de acerca de las funciones, las derivadas y las integrales en 1 y más variables
* Brindar herramientas para su cálculo en Python
* Familiarizarse y entender las diferencias entre las herramientas para calcular derivadas de manera simbólica y de manera numérica
* Familiarizarse con la idea de integración numérica usando la suma de Riemann

¿Qué **NO** busca esta clase?

* Cubrir completamente derivadas ni todos los conceptos que se discuten en esta notebook. Para esto se necesitaría al menos uno o dos curso de Cálculo o Análisis Matemático

Esta clase es una adaptación del precurso de la escuela de [Computational Neuroscience](https://compneuro.neuromatch.io/) de [Neuromatch Academy](https://academy.neuromatch.io/) y del material del libro [Hands-on Deep Learning with Scikit-Learn, Tensorflow and Keras](https://github.com/ageron/handson-ml2).

Dejaremos algunas referencias generales al final para que puedan profundizar en cada uno de los temas según los conocimientos de cada uno/una de ustedes. También utilizaremos (*) en el nombre de las sección que son optitivas debido a que su uso en DL es bastante poco relevante.

---
# Configuracion 

## Instalar dependencias

In [1]:
#@title Imports

import numpy as np
import scipy.optimize as opt  # import root-finding algorithm
import sympy as sp  # Python toolbox for symbolic maths
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # Toolbox for rendring 3D figures
from mpl_toolkits import mplot3d  # Toolbox for rendring 3D figures

## Configurar las figuras

In [2]:
# @title Settings
import ipywidgets as widgets  # interactive display
from ipywidgets import interact
#%config InlineBackend.figure_format = 'retina'
import seaborn as sns
import matplotlib as mpl
import matplotlib.animation as animation
mpl.rc('animation', html='jshtml')
sns.set_theme(style='darkgrid')
my_layout = widgets.Layout()

fig_w, fig_h = 9, 6
my_fontsize = 14
my_params = {'axes.labelsize': my_fontsize,
          'axes.titlesize': my_fontsize,
          'figure.figsize': [fig_w, fig_h],
          'font.size': my_fontsize,
          'legend.fontsize': my_fontsize-2,
          'lines.markersize': 8.,
          'lines.linewidth': 2.,
          'xtick.labelsize': my_fontsize-1,
          'ytick.labelsize': my_fontsize-1}

plt.rcParams.update(my_params)

## Otras funciones

In [3]:
# Funciones de figuras
def move_sympyplot_to_axes(p, ax):
    backend = p.backend(p)
    backend.ax = ax
    backend.process_series()
    backend.ax.spines['right'].set_color('none')
    backend.ax.spines['bottom'].set_position('zero')
    backend.ax.spines['top'].set_color('none')
    plt.close(backend.fig)


def plot_functions(function, show_derivative, show_integral):

  # For sympy we first define our symbolic variable
  x, y, z, t, f = sp.symbols('x y z t f')

  # We define our function
  if function == 'Lineal':
    f = -2*t
    name = r'$-2t$'
  elif function == 'Cuadrática':
    f =  t**2
    name = r'$t^2$'
  elif function == 'Exponencial':
    f =  sp.exp(t)
    name = r'$e^t$'
  elif function == 'Seno':
    f =  sp.sin(t)
    name = r'$sin(t)$'
  elif function == 'Sigmoidea':
    f = 1/(1 + sp.exp(-(t-5)))
    name = r'$\frac{1}{1+e^{-(t-5)}}$'

  if show_derivative and not show_integral:
    # Calculate the derivative of sin(t) as a function of t
    diff_f = sp.diff(f)
    print('Derivada de', f, 'es ', diff_f)

    p1 = sp.plot(f, diff_f, show=False)
    p1[0].line_color='r'
    p1[1].line_color='b'
    p1[0].label='Función'
    p1[1].label='Derivada'
    p1.legend=True
    p1.title = 'Función = ' + name + '\n'
    p1.show()
  elif show_integral and not show_derivative:

    int_f = sp.integrate(f)
    int_f = int_f - int_f.subs(t, -10)
    print('Integral de ', f, 'es ', int_f)


    p1 = sp.plot(f, int_f, show=False)
    p1[0].line_color='r'
    p1[1].line_color='g'
    p1[0].label='Función'
    p1[1].label='Integral'
    p1.legend=True
    p1.title = 'Función = ' + name + '\n'
    p1.show()


  elif show_integral and show_derivative:

    diff_f = sp.diff(f)
    print('Derivada de', f, 'es ', diff_f)

    int_f = sp.integrate(f)
    int_f = int_f - int_f.subs(t, -10)
    print('Integral de', f, 'es ', int_f)

    p1 = sp.plot(f, diff_f, int_f, show=False)
    p1[0].line_color='r'
    p1[1].line_color='b'
    p1[2].line_color='g'
    p1[0].label='Function'
    p1[1].label='Derivative'
    p1[2].label='Integral'
    p1.legend=True
    p1.title = 'Function = ' + name + '\n'
    p1.show()

  else:

    p1 = sp.plot(f, show=False)
    p1[0].line_color='r'
    p1[0].label='Function'
    p1.legend=True
    p1.title = 'Function = ' + name + '\n'
    p1.show()


def plot_alpha_func(t, f, df_dt):

  plt.figure()
  plt.subplot(2,1,1)
  plt.plot(t, f, 'r', label='PSP')
  plt.xlabel('')
  plt.ylabel('Voltaje')
  plt.legend()
  

  plt.subplot(2,1,2)
  plt.plot(t, df_dt, 'b', label='Derivada del PSP')
  #plt.title('Derivative of alpha function')
  plt.xlabel('Tiempo')
  plt.ylabel('df/dt')
  plt.legend()


def plot_charge_transfer(t, PSP, numerical_integral):

  fig, axes = plt.subplots(1, 2)

  axes[0].plot(t, PSP)
  axes[0].set(xlabel = 't', ylabel = 'PSP')

  axes[1].plot(t, numerical_integral)
  axes[1].set(xlabel = 't', ylabel = 'Charge Transferred')

# adapatada de hands-on
def show(axis="equal", ax=None, title=None, xlabel="$x$", ylabel="$y$"):
    ax = ax or plt.gca()
    ax.axis(axis)
    #ax.grid()
    ax.set_title(title, fontsize=14)
    ax.set_xlabel(xlabel, fontsize=14)
    ax.set_ylabel(ylabel, fontsize=14, rotation=0)
    ax.axhline(y=0, color='k')
    ax.axvline(x=0, color='k')

---
# 1 - ¿Que es el cáculo? Funciones, derivadas e integrales, una interpretación geométrica

El cálculo es una parte de las matemáticas que se ocupa del **cambio continuo** de un objeto en particular que son las **funciones**, es decir que estudia que sucede con las distintas variaciones que tienen las funciones. Hay dos ramas del cálculo: cálculo diferencial y cálculo integral, y comenzaremos con una introducción al primero.

Recordemos que una **función** $f$ en matemática es un tipo de relación entre dos conjuntos, el **dominio** y el **codominio**, en nuestro caso esos conjuntos serán los [números reales](https://es.wikipedia.org/wiki/N%C3%BAmero_real) denotado con $\mathbb{R}$, pero pueden ser otros conjuntos. Para denotar de donde y hacia donde llega la función solemos escribir:

\begin{align}
f: \mathbb{R} &\rightarrow \mathbb{R}\\
\mathcal{x} &\mapsto f(x)
\end{align}

Esto quiere decir que $f$ toma como entrada un número real $x$ y devuelve otro número real reales como salida. Podemos hacer una analogía con una máquina que recibe una entrada y devuelve algo transformado. Sin embargo, no cualquier relación es un función sino que como requisito, **para cada valor de entrada, la función le asigna un único valor de salida**.

In [4]:
# ejemplos de función en python
def mi_fun(x):
  y = x**2 - 2
  return y
print(mi_fun(3))

# no necesitamos necesariamente usar def para armar una función sino que podemos
# usar funciones lambda o anónimas
f = lambda x: x**2 -2
print(f(3))

7
7


La derivada de una funcion $f(x)$ es una nueva funcion que de alguna manera cauntifica como varia la funcion con respecto a su variable $t$. Existen muchas maneras de notar la derivada de $f$, en funciones de una variable de suele usar $f'(t)$ pero tambien (*disclaimer*: para facilitar esta explicacion asurimeros que esta derivada existe en todo el dominio de $f$ lo cual podria no suceder):

\begin{equation}
\partial_x f(x) = \frac{d(f(x))}{dx} = \frac{df}{dx} (x): \mathbb{R} \rightarrow \mathbb{R}
\end{equation}

Esta derivada captura que tan sensible es una funcion a cambios leves en la entrada para diferentes rangos de entradas. Geometricamente, la derivada de una funcion de una variable en cierta entrada ea la **pendiente** de una recta que es tangente a la funcion en la salida de esa funcion. Retomaremos esta idea en breve.